# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Mhamed Maatou

GitHub link: https://github.com/Mhamedben/M4JSONNLP.git

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [31]:
import requests
import json

# Function to search for song lyrics via the lyrics.ovh API
def get_lyrics(artist, title):
    url = f"https://api.lyrics.ovh/v1/{artist}/{title}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()  # Returns the lyrics data as a dictionary
    else:
        return {"error": "Lyrics not found"}

# Save lyrics to a JSON file
def save_lyrics_to_file(lyrics_data, filename="lyrics_data.json"):
    with open(filename, 'w') as json_file:
        json.dump(lyrics_data, json_file, indent=4)

# Load lyrics from a JSON file
def load_lyrics_from_file(filename="lyrics_data.json"):
    try:
        with open(filename, 'r') as json_file:
            return json.load(json_file)
    except FileNotFoundError:
        print(f"The file {filename} does not exist.")
        return None


# Fetch lyrics from the API
lyrics = get_lyrics(artist_name, song_title)

# Save the lyrics to a JSON file
save_lyrics_to_file(lyrics, "lyrics_data.json")
print("Lyrics saved to lyrics.json")



Lyrics saved to lyrics.json


2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [45]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Load spaCy model
nlp_model = spacy.load('en_core_web_sm')

# Add sentiment analysis pipeline
nlp_model.add_pipe("spacytextblob", last=True)

# Open and load the JSON file
with open('lyrics.json', 'r') as file:
    song_data = json.load(file)

# Retrieve lyrics from dictionary
song_lyrics = song_data.get('lyrics', '')

# Perform sentiment analysis
doc_analysis = nlp_model(song_lyrics)
sentiment_score = doc_analysis._.blob.polarity  # Get the polarity

# Display the lyrics and the sentiment score
print("Song Lyrics:\n", song_lyrics)
print("\nSentiment Score:", sentiment_score)

# Interpret the sentiment
if sentiment_score > 0:
    print("The lyrics are positive.")
elif sentiment_score < 0:
    print("The lyrics are negative.")
else:
    print("The lyrics are neutral.")

ModuleNotFoundError: No module named 'spacy'

3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [ ]:
import requests

def get_and_save_lyrics(artist, song, filename):
    # Construct the API URL
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    
    try:
        # Send a GET request to the API
        response = requests.get(url)
        
        # Check if the response is successful (status code 200)
        if response.status_code == 200:
            data = response.json()
            lyrics = data.get("lyrics", "Lyrics not found.")
            
            # Write the lyrics to the specified file
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(lyrics)
            print(f"Lyrics for {song} by {artist} have been saved to {filename}.")
        
        else:
            print(f"Error: Unable to fetch lyrics for {song} by {artist}. Status code {response.status_code}.")
    
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")

# Test the function by fetching lyrics for four songs
songs_to_test = [
    ("The Beatles", "Hey Jude", "hey_jude_lyrics.json"),
    ("Adele", "Hello", "hello_lyrics.json"),
    ("Ed Sheeran", "Shape of You", "shape_of_you_lyrics.json"),
    ("Queen", "Bohemian Rhapsody", "bohemian_rhapsody_lyrics.json")
]

for artist, song, filename in songs_to_test:
    get_and_save_lyrics(artist, song, filename)

Lyrics for Hey Jude by The Beatles have been saved to hey_jude_lyrics.json.
Lyrics for Hello by Adele have been saved to hello_lyrics.json.
Lyrics for Shape of You by Ed Sheeran have been saved to shape_of_you_lyrics.json.
Lyrics for Bohemian Rhapsody by Queen have been saved to bohemian_rhapsody_lyrics.json.


4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [ ]:
from textblob import TextBlob

def analyze_sentiment_from_file(filename):
    """
    This function takes the name of a file containing song lyrics,
    loads the lyrics, and returns the polarity score of the text.
    """
    try:
        # Load the lyrics from the file
        with open(filename, 'r', encoding='utf-8') as file:
            lyrics = file.read()
        
        # Perform sentiment analysis
        blob = TextBlob(lyrics)
        polarity = blob.sentiment.polarity
        
        return polarity
    
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
        return None

# List of the files created in the previous step
files_to_analyze = [
    "hey_jude_lyrics.json",
    "hello_lyrics.json",
    "shape_of_you_lyrics.json",
    "bohemian_rhapsody_lyrics.json"
]

# Analyzing sentiment for each song file and printing the polarity score
for file in files_to_analyze:
    polarity = analyze_sentiment_from_file(file)
    if polarity is not None:
        print(f"The polarity score for the lyrics in {file} is: {polarity}")

The polarity score for the lyrics in hey_jude_lyrics.json is: 0.13194444444444445
The polarity score for the lyrics in hello_lyrics.json is: -0.14109195402298852
The polarity score for the lyrics in shape_of_you_lyrics.json is: 0.31929292929292935
The polarity score for the lyrics in bohemian_rhapsody_lyrics.json is: -0.0029495701058200783
